In [4]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister,ClassicalRegister
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2,RealAmplitudes

from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from itertools import product
from scipy.optimize import minimize

In [5]:
noise = False 
n_qubits = 4
n_reps = 5
#nShots = 2**16
nShots = int(1e5)
print("nShots = ", nShots)


nShots =  100000


In [12]:
class VQE():
    
    def __init__(self,n_qubits = 4, n_reps = 5,shots=2**15,seed=None,noise = None,w = None):
        self.n_qubits = n_qubits
        self.n_reps = n_reps
        self.shots = shots
        self.seed = seed
        self.noise = noise
        self.H = None
        self.loss_history = []
        self.sampler = None
        self.estimator = None
        
        
        self.qc = QuantumCircuit(n_qubits)
        ansatz = RealAmplitudes(num_qubits=n_qubits, reps=n_reps, parameter_prefix='w')
        parameters = list(ansatz.parameters)
        self.qc.compose(ansatz, inplace=True)
        self.isa_qc = None
        if w!= None:
            self.w = w
        else:
            self.w = np.random.rand(len(self.qc.parameters))*2*np.pi
        
    def transpile(self):

        aer = AerSimulator()

        if self.noise:
            service = QiskitRuntimeService()
            real_backend = service.backend("ibm_kawasaki")
            aer = AerSimulator.from_backend(real_backend)
            self.estimator = Estimator(mode=aer)
            self.sampler = Sampler(mode = aer)
            pm = generate_preset_pass_manager(backend=aer, optimization_level=3)

        else:
            self.estimator = Estimator( mode = aer)
            self.sampler = Sampler( mode = aer)
            pm = generate_preset_pass_manager(backend=aer, optimization_level=3)

            self.isa_qc = pm.run(self.qc) 
            
    def getH(self):
        ob_list = []
        for i in range(1,n_qubits,1):
            ob = SparsePauliOp.from_list([("I"*(i ) + "Z" + "I"* ( n_qubits - i-1), 1)])
            ob = ob.apply_layout(layout=self.isa_qc.layout)
            ob_list.append(ob)

            ob = SparsePauliOp.from_list([("I"*(i ) + "X" + "I"* ( n_qubits - i-1), 1)])
            ob = ob.apply_layout(layout=self.isa_qc.layout)
            ob_list.append(ob)
        hamiltonian = sum(ob_list[1:], ob_list[0]) 
        self.H = hamiltonian

        
    def loss(self,w):
        
        result = self.estimator.run([(self.isa_qc,self.H,w)],precision = 0.005) # precision = 1 to sample only one time.
        evs = result.result()[0].data.evs
        return evs
    
    def train(self, maxiter=100):
        w = self.w

        def callback(w):
            current_loss = self.loss(w)
            self.loss_history.append(current_loss)

        res = minimize(self.loss, w, method='COBYLA', callback=callback, options={'maxiter': maxiter})
        self.w = res.x
        return res.x



In [ ]:
vqe = VQE()
vqe.transpile()
vqe.getH()
vqe.train()

In [ ]:
plt.plot(vqe.loss_history,label = "VQE")
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('VQE COBYLA')